# Testing Shaplet Generation

In [1]:
import shaplet_util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
sample = pd.read_csv('../../data/144/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)

In [3]:
plt.close()
plt.plot(sample['x'])

In [4]:
walking = sample.iloc[3000:19000]
walking = walking.reset_index()

In [5]:
period = util.find_single_peak_dist(walking, window_size=5000, step=100)
walking.index.size

16000

In [6]:
plt.close()
plt.plot(period['upper'], ".")
plt.xlabel('Window Index')
plt.ylabel('Average Peak Distance')

In [7]:
period['upper'].median()

99.0

In [8]:
plt.close()
plt.plot(period['lower'], ".")
plt.xlabel('Window Index')
plt.ylabel('Average Peak Distance')

In [9]:
period['lower'].median()

64.5

In [10]:
shaplet_len = period['upper'].median()

In [13]:
def dist(row):
    return row.apply(row_dist, args=(row.name,))

def row_dist(column, row):
    test_shaplet =  walking.loc[int(row):int(row)+shaplet_len].reset_index()
    ts_slice = walking.loc[int(column):int(column)+shaplet_len].reset_index()
    return np.linalg.norm(test_shaplet['x']- ts_slice['x'])

def set_column(x):
    return x.fillna(x.name)

In [14]:
#df = pd.DataFrame(data=[], index=np.arange(0,1000,20), columns=np.arange(0,1000,1))
#df = df.apply(set_column, axis=0)

In [15]:
#df = df.apply(dist, axis=1)

In [26]:
walking_test=walking.iloc[0:200]
plt.close()
plt.plot(walking_test['x'])

In [27]:
df = pd.DataFrame(data=[], index=np.arange(0,walking_test['x'].index.size-shaplet_len,20), columns=walking_test.index)
df.index = df.index.map(int)

In [28]:
df = df.apply(set_column, axis=0)

In [29]:
euclidean_dist_df = df.apply(dist, axis=1)

In [44]:
def get_best(row):
    num_best = int(row.size * .1)
    sorted_row = row.sort_values().reset_index()
    return sorted_row.loc[0:num_best, row.name]
euclidean_dist_df.apply(get_best, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,0.726429,1.120096,1.168412,1.322799,1.395313,1.403245,1.442945,1.489713,1.511575,...,1.729104,1.765334,1.813231,1.969101,1.987912,2.000740,2.022080,2.034052,2.042686,2.077869
20,0.0,0.704436,0.709277,1.212423,1.241124,1.357277,1.367282,1.436211,1.489274,1.789241,...,1.940255,1.945184,1.953712,1.984817,2.029135,2.098397,2.128573,2.130646,2.139721,2.140785
40,0.0,0.690162,0.720725,1.085245,1.200825,1.263421,1.314795,1.422253,1.543562,1.607948,...,1.965146,1.995104,2.065103,2.082156,2.086172,2.140006,2.160943,2.212895,2.217679,2.218609
60,0.0,0.613052,0.617292,1.036865,1.075811,1.171196,1.185877,1.238484,1.314049,1.569388,...,1.654920,1.685230,1.918718,1.966770,1.985053,1.996876,2.023013,2.042257,2.062036,2.081699
80,0.0,0.595510,0.597162,1.075884,1.132918,1.145581,1.154027,1.219089,1.349381,1.499094,...,1.641233,1.648659,1.827522,1.953540,1.992399,2.028444,2.040167,2.051798,2.060760,2.135674
100,0.0,0.570681,0.574704,1.095831,1.107825,1.123160,1.133417,1.165838,1.221661,1.232944,...,1.442753,1.483398,1.562225,1.574745,1.718467,1.804241,1.813231,1.970122,1.970540,2.029484


In [45]:
euclidean_dist_df.to_csv("144_candidate_shaplets")